In [1]:
!pip install -U bitsandbytes -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.5 MB/s eta 0:00:00


In [2]:
!pip install decord

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 106.2 MB/s eta 0:00:00


In [2]:
!pip install --upgrade accelerate diffusers transformers optimum[openvino]

  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.30.0
    Uninstalling accelerate-0.30.0:
      Successfully uninstalled accelerate-0.30.0


In [5]:
!pip install

ERROR: You must give at least one requirement to install (see "pip help install")


In [4]:
!pip install  requests==2.31.0 transformers==4.43.0 accelerate==0.30.0 #torch==2.3.0 torchvision==0.18.0 # flash_attn==2.5.8 Pillow==10.3.0 --no-build-isolation

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.3, but you have requests 2.31.0 which is incompatible.


In [6]:
!pip install numpy==2.2.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 105.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.5 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.5 which is incompatible.


In [7]:
from PIL import Image
import requests
from transformers import AutoModelForCausalLM, AutoProcessor
import torch # Explicitly import torch for torch_dtype

# Ensure you have a CUDA-enabled GPU available
if not torch.cuda.is_available():
    print("CUDA is not available. This script is intended to run on a CUDA-enabled GPU.")
    print("If you have a CUDA GPU, ensure PyTorch is installed with CUDA support.")
    exit()

model_id = "microsoft/Phi-3.5-vision-instruct"

# --- System Prompt for Full Text Extraction ---
system_prompt_content = """You are an expert Optical Character Recognition (OCR) assistant.
Your sole and critical task is to meticulously extract ALL text, numbers, and symbols visible in the provided image.
Transcribe the text exactly as it appears on the document, character by character.
You MUST ONLY output the extracted text. Do NOT add any introductory phrases, explanations, summaries, confidence scores, meta-comments, or any text whatsoever that is not directly read from the image.
Be extremely thorough and do not miss any textual element, no matter how small, faint, or seemingly insignificant.
"""

print("Loading model... This may take a while.")
try:
    # --- FIX 1: Changed flash_attention_2 to eager ---
    # This will resolve the ImportError if flash-attn is not installed.
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="cuda",
        trust_remote_code=True,
        torch_dtype="auto",
        _attn_implementation='eager' # Use 'eager' if flash_attn is not installed/working
    )
    print("Model loaded successfully.")
except ImportError as e:
    print(f"ImportError during model loading: {e}")
    print("This usually means your 'transformers' library is outdated or a dependency is missing.")
    print("Please try: pip install --upgrade transformers accelerate")
    print("If specifically for flash_attention_2, ensure 'flash-attn' is installed: pip install flash-attn --no-build-isolation")
    print("Switched _attn_implementation to 'eager' as a fallback.")
    exit()
except Exception as e:
    print(f"An error occurred while loading the model: {e}")
    exit()


print("Loading processor...")
processor = AutoProcessor.from_pretrained(
    model_id,
    trust_remote_code=True,
)
print("Processor loaded successfully.")

images = []
image_placeholders = []

# --- FIX 2: Load only ONE image ---
image_file_path = "/content/California -USA-_Front_f4dce8c3dfba3c893adafeec60cdd00d.jpg"
num_images_to_load = 1 # Changed from 19

print(f"Attempting to load the image from '{image_file_path}' {num_images_to_load} time(s)...")
# Simplified loading for a single image
try:
    print(f"Loading image from {image_file_path}...")
    image = Image.open(image_file_path)
    images.append(image.copy())
    image_placeholders.append(f"<|image_{len(images)}|>") # Will be <|image_1|>
    print(f"Successfully loaded the image.")
except FileNotFoundError:
    print(f"Error: Image file not found at '{image_file_path}'.")
    print(f"Please ensure the file exists at this path in your environment (e.g., upload to Colab's /content/).")
    exit()
except IOError as e:
    print(f"Error: Could not open or read image from '{image_file_path}'. Error: {e}")
    exit()
except Exception as e:
    print(f"Error: An unexpected error occurred while loading the image. Error: {e}")
    exit()

if not images:
    print("No image was loaded. Exiting.")
    exit()

print(f"Successfully loaded {len(images)} image(s).")
# --- END OF FIXED IMAGE LOADING ---


# Construct the string of image placeholders
image_placeholders_str = "".join([f"{ph}\n" for ph in image_placeholders])

# Define the user's task instruction
# Note: This user task instruction might be overridden by the strong system prompt for OCR.
# For pure OCR as per the system prompt, the user_task_instruction could be simplified
# or even an empty string if the model handles it correctly with the system prompt alone.
# However, keeping it as "Summarize" for now to see how the strong system prompt influences output.
# A better user_task_instruction for OCR would be "Extract all text from this image."
user_task_instruction = "Extract all text from this image." # Adjusted for OCR task

# Combine placeholders and task instruction for the user message content
user_content = image_placeholders_str + user_task_instruction

# Construct the messages list including the system prompt
messages = [
    {"role": "system", "content": system_prompt_content},
    {"role": "user", "content": user_content},
]

print("Applying chat template and tokenizing...")
try:
    prompt = processor.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = processor(text=prompt, images=images, return_tensors="pt").to("cuda:0")
    print("Inputs prepared and moved to CUDA.")
except Exception as e:
    print(f"Error during input processing: {e}")
    exit()

generation_args = {
    "max_new_tokens": 1024, # Increased for potentially longer OCR text
    "temperature": 0.0,
    "do_sample": False,
}

print("Generating response...")
try:
    generate_ids = model.generate(**inputs,
                                  eos_token_id=processor.tokenizer.eos_token_id,
                                  **generation_args
                                 )

    input_token_len = inputs['input_ids'].shape[1]
    generate_ids = generate_ids[:, input_token_len:]

    response = processor.batch_decode(generate_ids,
                                      skip_special_tokens=True,
                                      clean_up_tokenization_spaces=False)[0]

    print("\nGenerated Response:")
    print(response)

except torch.cuda.OutOfMemoryError:
    print("\nCUDA Out of Memory Error!")
    print("Try the following:")
    print("1. If you were loading many images previously, this should be less of an issue now.")
    print("2. Ensure 'eager' attention is working (it should be).")
    print("3. Close other applications using GPU memory.")
    print("4. Consider a GPU with more VRAM if consistently hitting this limit with complex tasks.")
    print("5. Reduce 'max_new_tokens' if the extracted text is expected to be very long and causes OOM.")
except Exception as e:
    print(f"An error occurred during generation: {e}")

Loading model... This may take a while.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/3.78k [00:00<?, ?B/s]

configuration_phi3_v.py:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3.5-vision-instruct:
- configuration_phi3_v.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

Processor loaded successfully.
Attempting to load the image from '/content/California -USA-_Front_f4dce8c3dfba3c893adafeec60cdd00d.jpg' 1 time(s)...
Loading image from /content/California -USA-_Front_f4dce8c3dfba3c893adafeec60cdd00d.jpg...
Successfully loaded the image.
Successfully loaded 1 image(s).
Applying chat template and tokenizing...


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


Inputs prepared and moved to CUDA.
Generating response...



Generated Response:
CALIFORNIA DRIVER LICENSE
DL I1234568
EXP 08/31/2018
CLASS C
END NONE
LN CARDHOLDER
FN IMA
2570 24TH STREET USA Template PSD
SACRAMENTO, CA 95818
DOB 08/31/1977
RSTR NONE
0831977
SEX M
HAIR BRN
EYES BRN
HGT 5'-05"
WGT 125 lb
DD 09/30/20106221/21FD/18
09/30/2010


# Phi-3.5-vision-instruct 8-Bit

In [1]:
from PIL import Image
import requests
from transformers import AutoModelForCausalLM, AutoProcessor
import torch # Explicitly import torch for torch_dtype

# Ensure you have a CUDA-enabled GPU available
if not torch.cuda.is_available():
    print("CUDA is not available. This script is intended to run on a CUDA-enabled GPU.")
    print("If you have a CUDA GPU, ensure PyTorch is installed with CUDA support.")
    exit()

model_id = "microsoft/Phi-3.5-vision-instruct"

# --- System Prompt for Full Text Extraction ---
system_prompt_content = """You are an expert Optical Character Recognition (OCR) assistant.
Your sole and critical task is to meticulously extract ALL text, numbers, and symbols visible in the provided image.
Transcribe the text exactly as it appears on the document, character by character.
You MUST ONLY output the extracted text. Do NOT add any introductory phrases, explanations, summaries, confidence scores, meta-comments, or any text whatsoever that is not directly read from the image.
Be extremely thorough and do not miss any textual element, no matter how small, faint, or seemingly insignificant.
"""

print("Loading model... This may take a while.")
print("Attempting to load the model in 8-bit precision...")
try:
    # --- MODIFICATION: Added load_in_8bit=True ---
    # This requires the `bitsandbytes` library to be installed.
    # You can install it with: pip install bitsandbytes
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="cuda", # Ensure model is on CUDA for 8-bit with bitsandbytes
        trust_remote_code=True,
        torch_dtype="auto", # torch_dtype can be auto or a specific float type like torch.float16
        load_in_8bit=True, # Enable 8-bit quantization
        _attn_implementation='eager' # Use 'eager' if flash_attn is not installed/working
    )
    print("Model loaded successfully in 8-bit.")
except ImportError as e:
    if "bitsandbytes" in str(e).lower():
        print(f"ImportError during model loading: {e}")
        print("8-bit quantization requires the 'bitsandbytes' library.")
        print("Please install it by running: pip install bitsandbytes")
    elif "flash_attention_2" in str(e).lower():
        print(f"ImportError during model loading: {e}")
        print("This usually means your 'transformers' library is outdated or a dependency is missing.")
        print("Please try: pip install --upgrade transformers accelerate")
        print("If specifically for flash_attention_2, ensure 'flash-attn' is installed: pip install flash-attn --no-build-isolation")
        print("Using _attn_implementation='eager' as a fallback for attention mechanism.")
    else:
        print(f"ImportError during model loading: {e}")
        print("This usually means your 'transformers' library is outdated or a dependency is missing.")
        print("Please try: pip install --upgrade transformers accelerate")
    exit()
except Exception as e:
    print(f"An error occurred while loading the model: {e}")
    exit()


print("Loading processor...")
processor = AutoProcessor.from_pretrained(
    model_id,
    trust_remote_code=True,
)
print("Processor loaded successfully.")

images = []
image_placeholders = []

# --- FIX 2: Load only ONE image ---
image_file_path = "/content/California -USA-_Front_f4dce8c3dfba3c893adafeec60cdd00d.jpg"
num_images_to_load = 1 # Changed from 19

print(f"Attempting to load the image from '{image_file_path}' {num_images_to_load} time(s)...")
# Simplified loading for a single image
try:
    print(f"Loading image from {image_file_path}...")
    # Check if the file exists before trying to open (optional, as open() will raise FileNotFoundError)
    # import os
    # if not os.path.exists(image_file_path):
    #     print(f"Error: Image file not found at '{image_file_path}'.")
    #     print(f"Please ensure the file exists at this path in your environment (e.g., upload to Colab's /content/).")
    #     exit()
    image = Image.open(image_file_path)
    images.append(image.copy()) # Use .copy() if you might modify the image later, otherwise not strictly necessary for this script
    image_placeholders.append(f"<|image_{len(images)}|>") # Will be <|image_1|>
    print(f"Successfully loaded the image.")
except FileNotFoundError:
    print(f"Error: Image file not found at '{image_file_path}'.")
    print(f"Please ensure the file exists at this path in your environment (e.g., upload to Colab's /content/).")
    exit()
except IOError as e:
    print(f"Error: Could not open or read image from '{image_file_path}'. Error: {e}")
    exit()
except Exception as e:
    print(f"Error: An unexpected error occurred while loading the image. Error: {e}")
    exit()

if not images:
    print("No image was loaded. Exiting.")
    exit()

print(f"Successfully loaded {len(images)} image(s).")
# --- END OF FIXED IMAGE LOADING ---


# Construct the string of image placeholders
image_placeholders_str = "".join([f"{ph}\n" for ph in image_placeholders])

user_task_instruction = "Extract all text from this image." # Adjusted for OCR task

# Combine placeholders and task instruction for the user message content
user_content = image_placeholders_str + user_task_instruction

# Construct the messages list including the system prompt
messages = [
    {"role": "system", "content": system_prompt_content},
    {"role": "user", "content": user_content},
]

print("Applying chat template and tokenizing...")
try:
    prompt = processor.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # For 8-bit models, ensure inputs are also on the correct device.
    # .to("cuda:0") or .to(model.device) should work.
    inputs = processor(text=prompt, images=images, return_tensors="pt").to(model.device)
    print(f"Inputs prepared and moved to {model.device}.")
except Exception as e:
    print(f"Error during input processing: {e}")
    exit()

generation_args = {
    "max_new_tokens": 1024, # Increased for potentially longer OCR text
    "temperature": 0.0,
    "do_sample": False,
}

print("Generating response...")
try:
    with torch.no_grad(): # Inference doesn't require gradient calculation
        generate_ids = model.generate(**inputs,
                                      eos_token_id=processor.tokenizer.eos_token_id,
                                      **generation_args
                                     )

    input_token_len = inputs['input_ids'].shape[1]
    generate_ids = generate_ids[:, input_token_len:]

    response = processor.batch_decode(generate_ids,
                                      skip_special_tokens=True,
                                      clean_up_tokenization_spaces=False)[0]

    print("\nGenerated Response:")
    print(response)

except torch.cuda.OutOfMemoryError:
    print("\nCUDA Out of Memory Error!")
    print("8-bit loading significantly reduces memory, but large images/long sequences can still cause OOM.")
    print("Try the following:")
    print("1. Ensure 'bitsandbytes' is correctly installed and working.")
    print("2. Close other applications using GPU memory.")
    print("3. Consider a GPU with more VRAM if consistently hitting this limit.")
    print("4. Reduce 'max_new_tokens' if the extracted text is expected to be very long and causes OOM.")
    print("5. If using a very high-resolution image, consider resizing it moderately.")
except Exception as e:
    print(f"An error occurred during generation: {e}")
    # If it's an error related to device placement with 8-bit, it might show up here.
    # e.g., "Expected all tensors to be on the same device"
    if "Expected all tensors to be on the same device" in str(e):
        print("This might be a device placement issue with 8-bit loading. Ensure model and inputs are on the same CUDA device.")
        print(f"Model device: {model.device}, Input device: {inputs['input_ids'].device}")

Loading model... This may take a while.
Attempting to load the model in 8-bit precision...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model loaded successfully in 8-bit.
Loading processor...


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/image_processing_auto.py:513: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(


Processor loaded successfully.
Attempting to load the image from '/content/California -USA-_Front_f4dce8c3dfba3c893adafeec60cdd00d.jpg' 1 time(s)...
Loading image from /content/California -USA-_Front_f4dce8c3dfba3c893adafeec60cdd00d.jpg...
Successfully loaded the image.
Successfully loaded 1 image(s).
Applying chat template and tokenizing...


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


Inputs prepared and moved to cuda:0.
Generating response...


/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")



Generated Response:
CALIFORNIA DRIVER LICENSE
DL I1234568
EXP 08/31/2018
CLASS C
END NONE
LN CARDHOLDER
FN IMA
2570 24TH STREET USA Template PSD
SACRAMENTO, CA 95818
DOB 08/31/1977
RSTR NONE
0831977
SEX M
HAIR BRN
EYES BRN
HGT 5'-05"
WGT 125 lb
DD 09/30/2010622121FD/18
09/30/2010



# 4-Bit

In [4]:
from PIL import Image
import requests # Not strictly needed for local images, but kept
from optimum.intel.openvino import OVModelForVisualCausalLM
from transformers import AutoProcessor, TextStreamer
import time # For timing execution

# The system prompt content to be incorporated
system_prompt_content = """You are an expert Optical Character Recognition (OCR) assistant.
Your sole and critical task is to meticulously extract ALL text, numbers, and symbols visible in the provided image.
Transcribe the text exactly as it appears on the document, character by character.
You MUST ONLY output the extracted text. Do NOT add any introductory phrases, explanations, summaries, confidence scores, meta-comments, or any text whatsoever that is not directly read from the image.
Be extremely thorough and do not miss any textual element, no matter how small, faint, or seemingly insignificant.
"""

model_id = "OpenVINO/Phi-3.5-vision-instruct-int4-ov"

print(f"Loading processor for {model_id}...")
load_start_time = time.time()
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)
print(f"Loading OVModel: {model_id}...")
ov_model = OVModelForVisualCausalLM.from_pretrained(model_id, trust_remote_code=True)
load_end_time = time.time()
print(f"Model and processor loading time: {load_end_time - load_start_time:.2f} seconds\n")


user_ocr_prompt = "<|image_1|>\n" # Image placeholder in user prompt

chat_messages = [
    {"role": "system", "content": system_prompt_content},
    {"role": "user", "content": user_ocr_prompt}
]

image_path = "/content/California -USA-_Front_f4dce8c3dfba3c893adafeec60cdd00d.jpg"
try:
    image = Image.open(image_path)
    # Ensure image is in RGB format if it has an alpha channel, as many models expect 3 channels
    if image.mode == 'RGBA':
        image = image.convert('RGB')
except FileNotFoundError:
    print(f"Error: The image file was not found at {image_path}")
    print("Please ensure the file exists and the path is correct (e.g., uploaded to Colab).")
    exit()
except Exception as e:
    print(f"Error opening image: {e}")
    exit()

# 1. Apply chat template to format messages into a single prompt string
# This correctly incorporates system prompts, user messages, and image tokens,
# and adds the necessary prompt for the model to start generating a response.
print("Formatting chat messages into prompt string...")
prompt_for_processor = processor.tokenizer.apply_chat_template(
    chat_messages,
    tokenize=False,  # We want a string, tokenization will happen in the next step
    add_generation_prompt=True  # Crucial for instructing the model to generate a response
)
# For debugging, you can print the formatted prompt:
# print(f"--- Formatted prompt for processor ---\n{prompt_for_processor}\n------------------------------------")

# 2. Process the formatted string prompt and the image using the processor
print("Preprocessing inputs (text prompt + image)...")
preprocess_start_time = time.time()
inputs = processor(
    text=prompt_for_processor,
    images=image,
    return_tensors="pt"
    # The processor's tokenizer will handle tokenization and adding special tokens (like BOS)
    # based on its configuration when given a string input.
)
preprocess_end_time = time.time()
preprocess_duration = preprocess_end_time - preprocess_start_time
print(f"Preprocessing time: {preprocess_duration:.2f} seconds")


generation_args = {
    "max_new_tokens": 512,
    "temperature": 0.0,
    "do_sample": False,
    "streamer": TextStreamer(processor.tokenizer, skip_prompt=True, skip_special_tokens=True)
}

print("\nSystem Prompt (effective):", system_prompt_content) # System prompt is part of `prompt_for_processor`
print("User Prompt (effective, part of chat):", user_ocr_prompt)
print("Image Path:", image_path)
print("\nOCR Output (streaming):")

generation_start_time = time.time()
generate_ids = ov_model.generate(**inputs,
                                 eos_token_id=processor.tokenizer.eos_token_id,
                                 **generation_args
                                )
generation_end_time = time.time()
generation_duration = generation_end_time - generation_start_time
# Streamer prints during generation.

# The following lines are for getting the response as a string variable.
response = ""
if generate_ids is not None:
    # Slice to get only the generated tokens (excluding the prompt)
    generated_tokens_only = generate_ids[:, inputs['input_ids'].shape[1]:]
    response = processor.batch_decode(generated_tokens_only,
                                      skip_special_tokens=True,
                                      clean_up_tokenization_spaces=False)[0]
    # Streamer already printed, but if you needed the variable:
    # print(f"\n--- Decoded response variable ---\n{response}\n---------------------------------")
else:
    print("\n\nNote: `generate_ids` is None, likely because a TextStreamer was used. The output was printed above.")


print(f"\n\n--- Execution Timing ---")
print(f"Preprocessing time: {preprocess_duration:.2f} seconds")
print(f"Generation (model inference) time: {generation_duration:.2f} seconds")
total_inference_time = preprocess_duration + generation_duration
print(f"Total inference (preprocessing + generation) time: {total_inference_time:.2f} seconds")

Loading processor for OpenVINO/Phi-3.5-vision-instruct-int4-ov...
Loading OVModel: OpenVINO/Phi-3.5-vision-instruct-int4-ov...
Model and processor loading time: 13.80 seconds

Formatting chat messages into prompt string...
Preprocessing inputs (text prompt + image)...
Preprocessing time: 0.06 seconds

System Prompt (effective): You are an expert Optical Character Recognition (OCR) assistant.
Your sole and critical task is to meticulously extract ALL text, numbers, and symbols visible in the provided image.
Transcribe the text exactly as it appears on the document, character by character.
You MUST ONLY output the extracted text. Do NOT add any introductory phrases, explanations, summaries, confidence scores, meta-comments, or any text whatsoever that is not directly read from the image.
Be extremely thorough and do not miss any textual element, no matter how small, faint, or seemingly insignificant.

User Prompt (effective, part of chat): <|image_1|>

Image Path: /content/California -US